In [1]:
from core.utils import Tibanna
from dcicutils import ff_utils


# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

env = 'data'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key={"default" : tibanna.ff_keys})

file_not_found = 0
deleted = 0
uploading = 0
success = 0
patch_problem = 0


files_resp = ff_utils.search_metadata('/search/?type=File&file_size=No+value&limit=all', ff_env = env)

print len(files_resp)

no_size = 0
counter = 0
for a_file in files_resp:
    if a_file['status'] == 'deleted':
        deleted += 1
        continue
    counter += 1
    if counter % 100 == 0:
        print counter,"+" ,deleted
    if a_file['status'] in ['uploading', 'to be uploaded by workflow']:
        uploading += 1
        continue
    # check if there is  no filesize
    if not a_file.get('file_size'):
        # decide on the bucket
        if 'FileProcessed' in a_file['@type']:
            bucket = tibanna.s3.outfile_bucket
        else:
            bucket = tibanna.s3.raw_file_bucket
            
        # check if file is in s3
        head_info = tibanna.s3.does_key_exist(a_file['upload_key'], bucket)
        if not head_info:
            print a_file['@id'], a_file['status']
            file_not_found += 1
            continue
        file_size = head_info['ContentLength']

        patch_data = {'file_size': file_size}
        try:
            ff_utils.patch_metadata(patch_data, obj_id=a_file['uuid'] ,ff_env=env)
            success += 1
        except Exception as e:
            print e
            print
            patch_problem += 1

print file_not_found, 'files missing from s3'
print patch_problem, 'files had patch problems'
print deleted, 'deleted files skipped'
print uploading, 'uploading files skipped'
print success, 'total files patched'

428
100 + 0
200 + 0
300 + 0
400 + 0
object 46e82a90-49e5-4c33-afab-9ec90d65cca1/4DNFIO67AFHV.fastq.gz not found on bucket elasticbeanstalk-fourfront-webprod-files
An error occurred (404) when calling the HeadObject operation: Not Found
/files-fastq/4DNFIO67AFHV/ uploaded
1 files missing from s3
0 files had patch problems
0 deleted files skipped
427 uploading files skipped
0 total files patched
